# Esteira de processamento de imagens para o modelo de classificação

Este notebook tem como objetivo gerar imagens para o modelo de classificação da Sprint 2. Portanto, ao final do arquivo estamos salvando imagens que contém talhões em pasta e em outra pasta os arquivos que não contém talhões.

Uma amostra das imagens geradas podem ser visualizadas em [`data -> SPRINT 2 -> MODELO DE CLASSIFICACAO`](../../../../data/SPRINT%202/MODELO%20DE%20CLASSIFICACAO/)

## Carregando as imagens e aplicando kmeans e sharpen

Abaixo estamos implementando o modelo de KMeans e Sharpen criados na Sprint 2. [Referência para o Modelo KMEANS](../../SPRINT%201/20240425%20-%20Implementacao%20KMeans%20para%20Reducao%20de%20Cor.ipynb)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import tqdm

In [ ]:
class KMeansImageProcessingPipeline:
	def __init__(self, base_dir):
		self.base_dir = base_dir

	def read_and_process_image(self, path):
		img = cv2.imread(path)
		img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
		return img

	def kmeans_image(self, img, k, attempts=10):
		# Redimensionar a imagem para um vetor 1D
		vectorized_img = img.reshape((-1,3))
		vectorized_img = np.float32(vectorized_img)

		# Definir critérios para o algoritmo K-means
		criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

		# Executar o algoritmo K-means
		ret, label, center = cv2.kmeans(vectorized_img, k, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
		center = np.uint8(center)

		# Recriar a imagem a partir dos clusters encontrados
		res = center[label.flatten()]
		result_image = res.reshape((img.shape))

		return result_image

	def sharpen_image(self, img):
		# Cria um kernel para afiar a imagem
		kernel = np.array([[-1,-1,-1],
											[-1,9,-1],
											[-1,-1,-1]])

		# Aplica o kernel na imagem usando filter2D
		sharpened_image = cv2.filter2D(img, -1, kernel)

		return sharpened_image

	def process_all_images(self, k, attempts=10):
		processed_images = []

		images = os.listdir(self.base_dir)

		for image_name in tqdm.tqdm(images):
			path = os.path.join(self.base_dir, image_name)
			img = self.read_and_process_image(path)

			# Aplica o algoritmo de k-means na imagem
			kmeans_img = self.kmeans_image(img, k, attempts)

			# Afia a imagem resultante
			sharpened_img = self.sharpen_image(kmeans_img)

			# Adiciona a imagem afiada à lista de retorno
			processed_images.append(sharpened_img)

		return processed_images

## Aplicando Data Augmentation

Aplicando o processo de Data Augmentation criado na Sprint 1. Esta implementação se diferencia pois compreendemos que não há necessidade de trabalhar com o Algoritmo de Clahe, implementado na Sprint 1, quando utilizamos imagens com mais de 1 dimensão, como no caso atual.

Também refatoramos o método `process_images` que agora possui a feature de salvar as imagens geradas.

[Referência para Implementação do Data augmentation da Sprint anterior](../../SPRINT%201/20240426%20-%20Imagens%20TIF%20e%20Data%20Augmentation.ipynb)

In [ ]:
from PIL import Image

class AugmentationImageProcessingPipeline:
    def __init__(self, output_dir):
        self.output_dir = output_dir
        # Cria o diretório de saída, se ele não existir
        os.makedirs(output_dir, exist_ok=True)

    @staticmethod
    def normalize_image(image):
        # Normaliza a imagem dividindo por 255 para que os valores fiquem no intervalo [0,1]
        return image / 255.0

    @staticmethod
    def resize_image(image, target_size):
        # Redimensiona a imagem para o tamanho desejado usando OpenCV
        return cv2.resize(image, target_size)

    @staticmethod
    def rotate_image(image, angle):
        # Rotaciona a imagem em um determinado ângulo
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)  # Calcula o centro da imagem
        M = cv2.getRotationMatrix2D(center, angle, 1.0)  # Cria a matriz de rotação
        # Aplica a transformação de rotação
        return cv2.warpAffine(image, M, (w, h))

    def crop_image(self, image, crop_size=(200, 200)):
        # Corta a imagem em sub-imagens menores de tamanho fixo
        crops = []
        img_height, img_width = image.shape[:2]

        # Percorre as dimensões da imagem original e realiza os cortes
        for row in range(0, img_height, crop_size[0]):
            for col in range(0, img_width, crop_size[1]):
                crop = image[row:row+crop_size[0], col:col+crop_size[1]]

                # Verifica se o tamanho do corte é válido antes de adicionar
                if crop.shape[0] == crop_size[0] and crop.shape[1] == crop_size[1]:
                    crops.append(crop)

        return crops

    def augment_images(self, image):
        # Gera diferentes variações da imagem original por rotação e espelhamento
        aug_images = []

        # Aplica rotações de 0, 90, 180, e 270 graus
        for angle in [0, 90, 180, 270]:
            rotated = self.rotate_image(image, angle)
            aug_images.append(rotated)
            # Espelha horizontalmente cada imagem rotacionada
            flipped = cv2.flip(rotated, 1)
            aug_images.append(flipped)

        return aug_images

    def process_and_save_images(self, images, target_size, crop=False):
        # Inicializa o contador global para salvar as imagens com nomes únicos
        global_count = 0

        print(f"Sinalizador de corte antes do processamento: {crop}, tipo: {type(crop)}")
        # Se o parâmetro `crop` for verdadeiro, executa a lógica de corte e augmentação
        if isinstance(crop, bool) and crop:
            for img in images:
                # Normaliza e redimensiona cada imagem
                norm_img = self.normalize_image(img)
                resized_img = self.resize_image(norm_img, target_size)
                # Corta a imagem redimensionada em patches menores
                cropped_imgs = self.crop_image(resized_img)

                for crop_img in cropped_imgs:
                    # Aplica augmentação a cada sub-imagem cortada
                    augmented_imgs = self.augment_images(crop_img)
                    for augmented_img in augmented_imgs:
                        # Converte para PIL e salva no diretório de saída
                        pil_img = Image.fromarray((augmented_img * 255).astype(np.uint8))
                        pil_img.save(os.path.join(self.output_dir, f"{global_count}.png"))
                        global_count += 1
        else:
            # Caso o corte não seja necessário, apenas redimensiona e aplica augmentação
            for img in images:
                norm_img = self.normalize_image(img)
                resized_img = self.resize_image(norm_img, target_size)
                augmented_imgs = self.augment_images(resized_img)
                for augmented_img in augmented_imgs:
                    # Converte para PIL e salva no diretório de saída
                    pil_img = Image.fromarray((augmented_img * 255).astype(np.uint8))
                    pil_img.save(os.path.join(self.output_dir, f"{global_count}.png"))
                    global_count += 1


## Aplicando a esteira nas imagens que não contém talhão


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Aplicando KMEANS E SHARPEN
base_dir = '/content/drive/MyDrive/desmatamento'
pipeline = KMeansImageProcessingPipeline(base_dir)
processed_images = pipeline.process_all_images(10)

In [ ]:
# Aplicando Data Augmentation e Salvando as Imagens
output_dir = '/content/drive/MyDrive/imagens_sem_talhao'
pipeline = AugmentationImageProcessingPipeline(output_dir)
pipeline.process_and_save_images(processed_images, target_size=(200,200))


## Aplicando a esteira nas imagens que contém talhão

In [ ]:
# Aplicando KMEANS E SHARPEN
base_dir = '/content/drive/MyDrive/imagens_com_talhao_raw'
pipeline = KMeansImageProcessingPipeline(base_dir)
processed_images = pipeline.process_all_images(10)

In [ ]:
# Aplicando Data Augmentation e Salvando as Imagens
output_dir = '/content/drive/MyDrive/imagens_com_talhao'
pipeline = AugmentationImageProcessingPipeline(output_dir)
pipeline.process_and_save_images(processed_images, target_size=(1200,1200), crop=True)